In [44]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from datetime import datetime
from meteostat import Point, Daily

In [45]:
# --- Load Dataset ---
# Import the dataset uploaded to the repo
url = "https://raw.githubusercontent.com/MaharLeika18/Data-Mining---Python/refs/heads/main/Final_Exam/retail_sales_dataset.csv"
data = pd.read_csv(url)

print("Data loaded successfully!")
print(data.head())

Data loaded successfully!
   Transaction ID        Date Customer ID  Gender  Age Product Category  \
0               1  2023-11-24     CUST001    Male   34           Beauty   
1               2  2023-02-27     CUST002  Female   26         Clothing   
2               3  2023-01-13     CUST003    Male   50      Electronics   
3               4  2023-05-21     CUST004    Male   37         Clothing   
4               5  2023-05-06     CUST005    Male   30           Beauty   

   Quantity  Price per Unit  Total Amount  
0         3              50           150  
1         2             500          1000  
2         1              30            30  
3         1             500           500  
4         2              50           100  


In [46]:
# --- Data Preprocessing ---
# One-hot encode all items 
transactions = data.groupby('Customer ID')['Product Category'].apply(list).values.tolist()

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

print("\nTransaction data prepared for analysis:")
print(df.head())


Transaction data prepared for analysis:
   Beauty  Clothing  Electronics
0    True     False        False
1   False      True        False
2   False     False         True
3   False      True        False
4    True     False        False


In [47]:
# Convert dates to datetime
data['Date'] = pd.to_datetime(data['Date'])
start = data['Date'].min()
end = data['Date'].max()
print(f"Date range: {start.date()} to {end.date()}")

Date range: 2023-01-01 to 2024-01-01


In [48]:
#Get historical weather data for Manila
location = Point(14.5995, 120.9842, 70)

weather_data = Daily(location, start, end)
weather_data = weather_data.fetch()

weather_data.head()

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2023-01-01,27.2,24.7,29.5,1.3,<NA>,<NA>,5.9,<NA>,1013.2,<NA>
2023-01-02,27.8,25.7,31.7,0.1,<NA>,<NA>,6.5,<NA>,1013.0,<NA>
2023-01-03,26.1,25.5,29.2,1.4,<NA>,<NA>,7.5,<NA>,1012.3,<NA>
2023-01-04,27.0,23.7,29.7,16.9,<NA>,<NA>,8.5,<NA>,1012.0,<NA>
2023-01-05,26.1,24.8,27.7,4.3,<NA>,<NA>,7.2,<NA>,1012.2,<NA>


In [49]:
# Add description to the data
def get_weather_description(row):
    temp = row['tavg']  # average temperature
    precip = row['prcp']  # precipitation
    
    if pd.isna(temp) or pd.isna(precip):
        return "Unknown"
    
    if precip > 10: 
        return "Heavy Rain"
    elif precip > 5:
        return "Rainy"
    elif precip > 1:
        return "Light Rain" 
    elif temp > 29:
        return "Sunny"
    elif temp > 27:
        return "Partly Cloudy"
    else: 
        return "Warm and Humid"

# Apply weather descriptions
weather_data['weather'] = weather_data.apply(get_weather_description, axis=1)

print("Weather text descriptions:")
print(weather_data['weather'].value_counts())
weather_data[['tavg', 'prcp', 'weather']].head(10)

Weather text descriptions:
weather
Light Rain        89
Partly Cloudy     79
Heavy Rain        66
Sunny             61
Warm and Humid    45
Rainy             26
Name: count, dtype: int64


,tavg,prcp,weather
time,,,
2023-01-01,27.2,1.3,Light Rain
2023-01-02,27.8,0.1,Partly Cloudy
2023-01-03,26.1,1.4,Light Rain
2023-01-04,27.0,16.9,Heavy Rain
2023-01-05,26.1,4.3,Light Rain
2023-01-06,26.0,5.9,Rainy
2023-01-07,26.7,0.0,Warm and Humid
2023-01-08,25.7,0.0,Warm and Humid
2023-01-09,26.0,0.0,Warm and Humid


In [50]:
# Create weather_data_reset for merging
weather_data_reset = weather_data.reset_index()
weather_data_reset.rename(columns={'time': 'Date'}, inplace=True)

print(f"Shape: {weather_data_reset.shape}")
display(weather_data_reset[['Date', 'weather']].head())

display(weather_data_reset)
# Merge 
data_with_weather = pd.merge(data, weather_data_reset[['Date','weather','tavg','prcp','wspd']], on='Date', how='left')

Shape: (366, 12)


,Date,weather
0,2023-01-01,Light Rain
1,2023-01-02,Partly Cloudy
2,2023-01-03,Light Rain
3,2023-01-04,Heavy Rain
4,2023-01-05,Light Rain


,Date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,weather
0,2023-01-01,27.2,24.7,29.5,1.3,<NA>,<NA>,5.9,<NA>,1013.2,<NA>,Light Rain
1,2023-01-02,27.8,25.7,31.7,0.1,<NA>,<NA>,6.5,<NA>,1013.0,<NA>,Partly Cloudy
2,2023-01-03,26.1,25.5,29.2,1.4,<NA>,<NA>,7.5,<NA>,1012.3,<NA>,Light Rain
3,2023-01-04,27.0,23.7,29.7,16.9,<NA>,<NA>,8.5,<NA>,1012.0,<NA>,Heavy Rain
4,2023-01-05,26.1,24.8,27.7,4.3,<NA>,<NA>,7.2,<NA>,1012.2,<NA>,Light Rain
...,...,...,...,...,...,...,...,...,...,...,...,...
361,2023-12-28,28.0,25.9,30.5,1.1,<NA>,<NA>,6.3,<NA>,1015.1,<NA>,Light Rain
362,2023-12-29,27.9,25.9,30.2,1.2,<NA>,<NA>,6.9,<NA>,1013.5,<NA>,Light Rain
363,2023-12-30,27.9,26.0,30.2,1.9,<NA>,<NA>,6.7,<NA>,1012.2,<NA>,Light Rain
364,2023-12-31,28.2,26.2,30.5,3.3,<NA>,<NA>,6.1,<NA>,1012.1,<NA>,Light Rain


In [51]:
# Export
data_with_weather.to_csv('retail_data_with_weather.csv', index=False)
display(data_with_weather)

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,weather,tavg,prcp,wspd
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,Partly Cloudy,28.4,0.8,7.0
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,Warm and Humid,26.9,0.0,9.3
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,Warm and Humid,26.5,0.6,6.5
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,Light Rain,29.8,1.8,7.3
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,Light Rain,31.4,1.1,9.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,2023-05-16,CUST996,Male,62,Clothing,1,50,50,Heavy Rain,29.6,15.0,10.4
996,997,2023-11-17,CUST997,Male,52,Beauty,3,30,90,Partly Cloudy,28.9,0.0,9.6
997,998,2023-10-29,CUST998,Female,23,Beauty,4,25,100,Sunny,30.0,0.5,6.7
998,999,2023-12-05,CUST999,Female,36,Electronics,3,50,150,Light Rain,28.8,1.6,6.9
